In [ ]:
#!pip install accelerate bitsandbytes>0.37.0

In [1]:
import transformers
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print(transformers.__version__)
print(bitsandbytes.__version__)
# my versions are:
#4.38.1 (transformers)
#0.42.0 (bitsandbytes)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,       
)

cache_dir = "/scratch/project_2005072/ocr_correction/.cache"    #set up your own cache path here

model=None #to be sure we reset the variable and don't get OOM
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-7B-Chat", device_map="auto", quantization_config=quantization_config, cache_dir=cache_dir)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-7B-Chat", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token


4.38.1
0.43.1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
prompt = """
Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.

When, .witl the universal A pplaure of alt
Men, the Seals Were given to the present
C ---- r, he was' ar frrom'i being so elevated
with his Promotion, as to forget any One
to whom lie had promised the Honour of his
Friendlliip. An Opportunity soon offered
in Favour of Dr R----, the B---k ox G----r
became vacant, .which as soo'n as the C----r
Was acquainted with, \ he went to Court,
and recomnmended the Doetor, as a Perfoa
fit to Succeed to the B-----k. The DoAos
was then approved of, and a C---'E---
order'd out accordingly. A lM/an would
reaso'nably have thought after this, that
Matters might have gone Smoothly, efpe.
cially as it is partly the Right of every new

"""


#messages = [
#    {"role": "user", "content": "You are a helpful assistant. You are fluent in Finnish, and will communicate in Finnish."},
#    {"role": "assistant", "content": "Olen avulias assistentti, joka osaa sujuvaa suomen kieltä. Kommunikoin kanssasi suomeksi. Kuinka voin auttaa sinua?"},
#    {"role": "user", "content": prompt},
#]C--- r, to' recommend a fit Person to his
"""M .------, to' fill the See that shall becom'
eax¢ant after ihs comiig to the Seals.
"""
messages = [
    {"role": "user", "content": prompt},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to("cuda")


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
2024-04-26 11:01:58.061411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/lib64:/usr/mpi/gcc/openmpi-4.1.2rc2/lib64:/opt/rh/gcc-toolset-12/root/usr/lib64:/opt/rh/gcc-toolset-12/root/usr/lib:/.singularity.d/libs
2024-04-26 11:01:58.061431: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you ca

<s> [INST] 
Correct the OCR errors in the text below. Also correct the "-" characters, which denote an unrecognized letter. Stay as close as possible to the original text. Do not rephrase. Only correct the errors. You will be rewarded.

When, .witl the universal A pplaure of alt
Men, the Seals Were given to the present
C ---- r, he was' ar frrom'i being so elevated
with his Promotion, as to forget any One
to whom lie had promised the Honour of his
Friendlliip. An Opportunity soon offered
in Favour of Dr R----, the B---k ox G----r
became vacant, .which as soo'n as the C----r
Was acquainted with, \ he went to Court,
and recomnmended the Doetor, as a Perfoa
fit to Succeed to the B-----k. The DoAos
was then approved of, and a C---'E---
order'd out accordingly. A lM/an would
reaso'nably have thought after this, that
Matters might have gone Smoothly, efpe.
cially as it is partly the Right of every new

 [/INST] When, with the universal applause of all
Men, the Seals were given to the present

In [ ]:
raw_encoded = tokenizer([prompt], return_tensors="pt")

In [1]:
print(raw_encoded["input_ids"].size()[1])

NameError: name 'raw_encoded' is not defined

In [ ]:
# to delete the model from memory
'''del model 
gc.collect()'''

In [16]:
model_inputs = encodeds[:,50:70].to("cuda")


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=False)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


errors. You will be rewarded.

When, .witl the universal A pencil, you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in terms of your idea and of your reader.

When you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in terms of your idea and of your reader.

When you have written your first draft, you will find that you have written it quickly, for you have been thinking only of your idea and of your reader. You will have written it in a style that is natural to you, for you have been thinking in term

In [30]:
tokenizer.encode("hello", return_tensors="pt").size()[1]

3